# 6월 27일 학습

### 챗봇 보완 및 추가 기능
- 어제 만든 챗봇에 기능을 추가해보고, keras 학습 모델을 이용해 개와 고양이를 분류해주는 프로그램을 만들어보기

- 개와 고양이를 분류한 후, OpenAI를 이용하여 추가적인 정보 전달 기능의 추가 등으로 확장할 수 있다!


- 실습 위주로 학습. 학습 파일 참고하기
- tensor_proj repository:
    - chatbot.py
    - chatbot_version2.py
    - st_app2.py
    - PBL2_3



### 챗봇 ver.2

In [ ]:
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

client = OpenAI(api_key=OPENAI_API_KEY)

#함수 정의
def get_today_date():
    today = datetime.now().strftime('%Y-%m-%d') #2025-06-27
    return today
#tools = []
tools = [
    {
        "type" : "function",
        "name" : "get_today_date",
        "description" : "오늘 날짜를 YYYY-MM-DD 형식의 문자열로 반환합니다. ",
        "parameters" : {
            "type" : "object",
            "properties" : {},
            "required" : []
        }
    }
]
st.title('ChatGPT Ver.2')

#세션 관리 객체. 세션에 키-값 형식으로 데이터를 저장하는 변수
#openai_model 저장 --> str, message(사용자가 요청한 메시지)--> []
if 'openai_model' not in st.session_state :
    #openai_model키 없으면 추가
    st.session_state.openai_model = 'gpt-4.1'

if 'messages' not in st.session_state:
    #message 키 없으면 추가 및 초기화
    st.session_state.messages = []

#기존의 메시지가 있다면 출력
for msg in st.session_state.messages:
    if msg.get('role') in ('user', 'assistant'):
        with st.chat_message(msg['role']):
            st.markdown(msg['content'])



#prompt --> 사용자 입력 창
if prompt := st.chat_input('메시지를 입력하세요'):
    #message --> [], 대화 내용 추가
    st.session_state.messages.append({
        "role": "user",
        "content": prompt
    })
        
    with st.chat_message('user'):
        st.markdown(prompt)
    
    #첫 번째 openai 요청 --> 함수 선택 --> 함수 결과를 이용해서 재 요청
    response = client.responses.create(
        model = st.session_state.openai_model,
        input = st.session_state.messages,
        tools = tools
    )

    #함수 호출 처리
    msg_content = None  #초기화. 최종 응답 결과
    tool_executed = False   #함수 호출 여부


    if response.output:
        for tool_call in response.output:
            print(f'Tool 호출: {tool_call}')
            # type=='function_call' and  name='get_today_date'
            if tool_call.type=='function_call' and tool_call.name=='get_today_date':
                print(f'function call : {tool_call.name}')
                args = json.loads(tool_call.arguments or "{}")
                result = get_today_date()
                tool_executed=True

                st.session_state.messages.append({
                    "type" : "function_call",
                    "call_id" : tool_call.call_id,
                    "name" : tool_call.name,
                    "arguments" : tool_call.arguments
                })

                st.session_state.messages.append(
                    {
                        "type" : "function_call_output",
                        "call_id" : tool_call.call_id,
                        "output" : result
                    }
                )
    
    if tool_executed : 
        response2 = client.responses.create(
            model=st.session_state.openai_model,
            input=st.session_state.messages,
            tools=tools
        )

        msg_content = getattr(response2, 'output_text', None)

        with st.chat_message('assistants'):
            st.markdown(msg_content)

            # #messages --> append
            # st.session_state.messages.append({
            #     "role" : "assistant",
            #     "content" : msg_content,
            # })
            #함수 호출 후 최종 결과 처리 끝
    #함수 호출이 없을 때
    else:
        msg_content = getattr(response, 'output_text', None)
        with st.chat_message('assistant'):
            st.markdown(msg_content)

    st.session_state.messages.append({
        "role" : "assistant",
        "content" : msg_content
    })

### keras 모델을 이용한 개/고양이 분류 챗봇


In [ ]:
# 모델 로드
def load_model():
    try:
        st.success('모델을 로드했습니다.')
        return tf.keras.models.load_model('cat_dog_classifier.keras')
    except:
        st.error('모델을 로드할 수 없습니다. 경로를 확인해주세요!')

model = load_model()

# 전처리 --> 사용자가 업로드한 이미지를 전처리한다.
def preprocess_image(image):
    try:
        image = image.resize((150, 150))  #150 by 150 크기로 모델을 학습시킴
        image = np.array(image) /255.0  #정규화

        if image.shape[-1]  != 3: #배열의 마지막 차원의 크기!.
            raise ValueError('이미지는 RGB 형식의 컬러 이미지만 처리가 가능합니다.')
        image = np.expand_dims(image, axis=0)   #(1, 150, 150, 3)
        return image
    except Exception as e:
        st.error(f'이미지 전처리중 문제가 발생했습니다.: {e}')
        return None



#UI
st.title('Cat/Dog 분류기')
st.write('이미지를 업로드 하면 개 또는 고양이를 판별합니다.')

uploadfile = st.file_uploader('이미지를 업로드하세요.', type=['jpg', 'png', 'jpeg'])

if uploadfile is not None:
    try:
        #이미지 로드 --> 이미지 파일을 이미지 객체로 변환
        image = Image.open(uploadfile)
        st.image(image, caption='업로드 한 이미지', use_column_width=True)

        preprocessed_image = preprocess_image(image)#이미지 전처리

        if preprocessed_image is not None:
            prediction = model.predict(preprocessed_image)
            print(prediction)

            #결과 표시
            if prediction [0][0] > 0.5 :
                st.success('이 이미지는 개로 분류되었습니다.')
            else:
                st.success('이 이미지는 고양이로 분류되었습니다.')
    except UnidentifiedImageError : #이미지 형식이 아닌 경우 발생
        st.error('이미지를 로드할 수 없습니다. 지원되지 않는 파일 형식입니다.')
    except Exception as e:
        st.error(f'예측 처리 중 오류 발생: {e}')

# Python Web Programing
- 웹 서비스를 위한 기능. 서비스를 서빙하기 위해 요청에 대해 응답하는 형식으로 통신을 한다.
- 웹 기반 소프트웨어로 사용하는 서버, Rest API, JSON등에 Flask, Django(MVC), Fast API(Rest API를 구현하는 데 최적화) 등을 사용한다.
- 우리나라에서는 자바, 자바스프링 프레임워크가 기본이다.


### Fast API
1. FastAPI
- fast api 사이트에서 정보를 얻을 수 있다.
- 고성능 웹 프레임워크: 파이썬 기반의 비동기 웹 프레임워크로 빠른 성능을 제공한다.
- ASGI 지원: 비동기 요청을 처리하고 **웹 소켓**(Rest API)과 같은 실시간 통신을 지원
- 자동화된 API 문서화: Swagger UI와 ReDoc을 통해 자동으로 API 문서를 생성
- Pydantic 통합: 데이터 유효성 검사와 데이터 모델링을 쉽게 처리
- 타입 힌트 기반: 파이썬의 타입 힌트를 활용하여 자동으로 데이터 검증
- 간단한 코드: 적은 양의 코드로도 강력한 기능을 구현할 수 있어 생산성이 높음

2. Uvicorn
    - 초고속 ASGI 서버 (Asynchronous Server Gateway Interface)
    - ASGI 표준 지원 (비동기 프로그래밍, 실시간 기능 처리)
        - --> 채팅, 알림 시스템과 같은 실시간 서비스를 구현에 유리
    - 간단한 실행: uvicorn main:app --reload
    - 자동 리로딩 지원: --reload 옵션

3. Pydantic
- Python에서 가장 널리 사용되는 데이터 유효성 검사 라이브러리 
    - --> 데이터 유효성 검사 자동 수행

- Python 3.8 이상에서 데이터가 어떻게 구성되어야 하는지 정의 
    - --> 데이터 모델링 : 클래스 기반 데이터 구조 정의

- Pydantic으로 데이터의 유효성을 검사
    - 타입 힌트 지원 : 직관적이고 명확한 데이터 정의
    - 자동 타입 변환 지원
    - FastAPI와의 통합 : 입력 데이터 검증에 유용

- 설치:
    - pip install fastapi
    - pip install "uvicorn[standard]"
    - pip install pydantic

- **배포까지 해보면 좋다!!**


### Flask
- 플라스크는 파이썬에서 가벼운 웹 애플리케이션을 만들기 위한 라이브러리를 제공하는 웹 프레임워크

- WSGI 툴킷과 jinja2 템플릿 엔진을 기반으로 하며, 플라스크는 마이크로 프레임워크로 간주된다.

- WSGI
    - 웹 서버 게이트웨이 인터페이스의 약어
    - 파이썬 웹 애플리케이션 개발을 위한 표준
    - 웹 서버와 웹 애플리케이션 간의 범용 인터페이스 명세로 간주

- Jinja2
    - Jinja2는 특정 데이터 소스와 템플릿을 결합하여 동적 웹 페이지를 렌더링하는 웹 템플릿 엔진


- route() 함수
    - Flask 클래스의 route() 함수는 연결된 함수의 URL 매핑을 정의
    - @app.route(rule, option)
        - rule: 함수와 연결된 URL을 나타낸다.
        - option: rule 객체와 관련된 매개변수 목록을 나타낸다.
        - 이 함수는 브라우저 창에 인쇄할 문자열을 반환하거나 서버에서 응답으로 HTML 파일을 반환하는 데 HTML 템플릿을 사용

- run 메서드
    - Flask 클래스의 run 메서드를 사용하여 로컬 개발 서버에서 flask 애플리케이션을 실행
    - app.run(host, port, debug, option)
        - host: 기본 호스트 이름은 127.0.0.1 또는 localhost
        - port: 서버가 수신 대기하는 포트 번호. 기본 포트 번호는 5000
        - debug: 기본값은 false. true로 설정하면 디버그 정보를 제공
        - option: 서버로 전달할 정보를 포함

### Flask App routing
- 앱 라우팅은 특정 URL을 특정 작업을 수행할 함수와 매핑하는 개념

- 용도: 주로 특정 페이지에 접근하기 위해 사용

- 작동 방식: 예를 들어, 첫 번째 애플리케이션에서는 URL('/')이 홈 함수와 연결되어 해당 함수가 반환하는 문자열이 웹 페이지에 표시

- 결과 표시: 특정 URL을 방문하면 연결된 함수의 결과가 브라우저 화면에 렌더링

# 파이썬 정리.

- 파이썬을 통해 자동화 업무, 또는 오피스 관련 라이브러리들을 사용하여 효율을 높일 수 있다.

- 판다스 라이브러리를 잘 사용해보자

- 분석/통계쪽 공부를 해두면 좋다
    - 특정 데이터에서 insight를 추출할 수 있는 것이 능력이다.

- 데이터를 다루는 연습이 중요하다.

- **기본적으로 데이터 수집을 하는 방법을 다양하게 알고 가공하는 기술을 기르자!**

    - 데이터 수집, 데이터 전처리, 기초 통계는 필수
    - 머신러닝 관련해서도 기초적인 것만 실습하였음. 공부하면서 규칙을 찾아내는 등의 분석 및 처리 방법을 잘 정리해두자

- **Hugging Face** 
    - 많은 모델들이 만들어져있다. --> 자동화의 도구로 처리해보는 연습을 할 수 있다.

- AI-Hub
    - 여러 데이터들을 사용할 수 있다.
    - AI에 사용할 수 있는 데이터셋들을 가지고 모델을 만들어 처리해볼 수 있다.

# 클래스, 객체지향
- 클래스 만들기
    - **일반화가 가능한가?**
    - 일반화의 개념부터 잡고 시작하기!
    - 생성자 필요: 초기화
    - 메서드(기능)

- 더 나아가 **일반화**를 할 수 있도록 하자! 어떤 틀을 만드는 것이 중요하다